In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [ ]:
from exp import nb_anchors_loss_metrics_std
from exp import nb_resnet_ssd_std

In [ ]:
import sys 
sys.path.append("..") 
from detect_symbol.exp import nb_databunch
from detect_symbol.exp import nb_resnet_ssd
from detect_symbol.exp import nb_anchors_loss_metrics

In [ ]:
sys.path.append("../detect_symbol") 
from detect_symbol.exp import nb_interpretation

In [ ]:
from detect_symbol.exp.nb_anchors_loss_metrics import get_y, idx_fromScoresHits

In [ ]:
from detect_symbol.exp.nb_interpretation import draw_rect, draw_text, draw_outline

In [ ]:
# export
import torch

In [ ]:
# export
from torch import tensor

In [ ]:
from fastai.basic_train import Learner

In [ ]:
# export
from matplotlib import pyplot as plt

In [ ]:
# export
from matplotlib import patches,patheffects

In [ ]:
# export
from IPython.core import debugger as idb

In [ ]:
from fastai.vision.image import Image

In [ ]:
from functools import partial

In [ ]:
# export
from math import ceil

# functions

### show_gt

In [ ]:
def show_gt(x,y):
    img = data.denorm(x)[0].permute(1,2,0)

    _,ax = plt.subplots(1,1,figsize=(15,15))
    ax.imshow(img)
    
    for bb,cat in zip(y[0][0],y[1][0]):
        bb = (bb+1)/2*776
        draw_rect(ax,bb,lw=1)
        
        cat = cat-1
        clas = classes[cat]
        draw_text(ax,bb[[3,0]],clas[:3],sz=10)

### show_gt_anchors

In [ ]:
def show_gt_anchors(x,y,device,n=10):
    '根据ground truch找到负责idxs，绘制这些idxs对应的anchor'
    _,ax = plt.subplots(1,1,figsize=(20,20))
    img = data.denorm(x)[0].permute(1,2,0)
    ax.imshow(img)
    #import pdb; pdb.set_trace()    
    idxs = idx_from_gt(*y,gaf=gaf)
    for idx in idxs:
        gv = gvs[idx]*x.shape[-1]
        draw_rect(ax,gv,lw=1,color='g')
        
        cx = gv[[0,2]].mean()
        cy = gv[[1,3]].mean()
        
        av = avs[idx]*x.shape[-1]
        av = av+tensor([cx,cy,cx,cy]).to(device)
        draw_rect(ax,av,lw=1)
        

        draw_text(ax,av[[3,0]],f'{idx}',sz=10)
    return idxs

###  idx_from_gt

In [ ]:
def idx_from_gt(*gt_batch, gaf):
    loss = 0
    cnt = 0
    for gt_bboxs in gt_batch[0]:
        keep = get_y(gt_bboxs)
        if keep.numel()==0: continue
          
        gt_bboxs = gt_bboxs[keep]
        
        gt_bboxs = (gt_bboxs + 1) / 2
        
        scores,hits = gaf.get_scores_hits(gt_bboxs)
        idx = idx_fromScoresHits(scores,hits)
        
        return idx

In [ ]:
def process_netout_std(txys,cats,confs,gaf):
    ts = torch.cat([txys],dim=-1)
    ts2 = torch.cat([txys,txys],dim=-1)
    ts.device
    boxs = gaf.t2b(ts, torch.arange(start = 0, end = ts.shape[-2], device=ts.device))
    
    confs = torch.sigmoid(confs).squeeze(-1)
    prbs,cats = torch.softmax(cats,dim=-1).max(dim=-1)
#     cats = cats + 1
    
    return boxs,confs,cats,prbs

In [ ]:
def netouts2preds_std(batchOut,gaf,composeConfPrb=True,filt_thres=0.9,ov_thres=0.2,despiteCat=True):
    '''
    处理一个batch
    '''
    batch_boxs = []
    batch_confs = []
    batch_cats = []
    batch_prbs = []
    batch_f1s = []
    
    for txys,confs,cats in zip(batchOut[0].detach(),batchOut[1].detach(),batchOut[2].detach()):
        boxs,confs,cats,prbs = process_netout_std(txys,cats,confs,gaf)
        
        f1s = 2*confs*prbs/(confs+prbs)
        if composeConfPrb:
            filt = torch.where(f1s>filt_thres)
        else:
            filt = torch.where(confs>filt_thres)
            
        boxs = boxs[filt]
        confs = confs[filt]
        cats = cats[filt]
        prbs = prbs[filt]
        f1s = f1s[filt]
        
        '''
        没有宽高。所以根本不需要nms
        if composeConfPrb:
            keep,cnt = nb_interpretation.nms(boxs, cats, f1s, ov_thres=ov_thres, despiteCat=True)
        else:
            keep,cnt = nb_interpretation.nms(boxs, cats, confs, ov_thres=ov_thres, despiteCat=True)            
        keep = keep[:cnt].long()        
        boxs = boxs[keep]
        confs = confs[keep]
        cats = cats[keep]
        prbs = prbs[keep]
        f1s = f1s[keep]
        '''
        
        batch_boxs += [boxs]
        batch_confs += [confs]
        batch_cats += [cats]
        batch_prbs += [prbs]
        batch_f1s += [f1s]
        
    return batch_boxs, batch_confs, batch_cats, batch_prbs, batch_f1s

In [ ]:
def show_top_pred(x,netOut,n=10):
    '''
    根据模型输出，处理得到boxs和confs，按照confs从大到小排序，绘制前n个的box.
    x的作用仅是为绘制原图；
    '''
    _,ax = plt.subplots(1,1,figsize=(20,20))
    img = data.denorm(x)[0].permute(1,2,0)
    ax.imshow(img)
    
    txys,confs,cats = netOut[0][0].detach(),netOut[1][0].detach(),netOut[2][0].detach()
    boxs,confs,cats,prbs = process_netout(txys,cats,confs,gaf)
    
    top_idx = confs.sort(descending=True)[1]
    for idx in top_idx[:n]:
        bb = boxs[idx]*x.shape[-1]
        #draw_rect(ax,bb,lw=1)
        w, h = 22, 22
        lw = 1
        patch = ax.add_patch(patches.Rectangle((bb[1],bb[0]), w, h
                    , fill=False, edgecolor='green', lw=lw))
        draw_outline(patch, lw=lw)

        clas = classes[cats[idx]]
        conf = confs[idx]
        info = '{}_{}_{:.2f}'.format(idx,clas[:3],conf*100)
        draw_text(ax,bb[[3,0]],info,sz=10)
    return top_idx[:n]

# test

## 准备好learner

In [ ]:
device = torch.device('cuda')

In [ ]:
#先用detect_symbol的数据集测试
data = nb_databunch.get_databunch(data_root = '../detect_symbol/data/ds_20200429', bs=4, device=device)
x,y = data.one_batch()

In [ ]:
gaf = nb_anchors_loss_metrics_std.GridAnchor_Funcs_std(fig_hw = (776,776)
                         , grids = [(49,49)]
                         , device = device)
gvs, avs = gaf.gvs, gaf.avs

In [ ]:
model = nb_resnet_ssd_std.get_resnet18_1ssd_std(num_classes = 17)

In [ ]:
fn = '../detect_symbol_bak20200427/data/ds_20200227/models/run_0.pth'
stat = torch.load(fn)

In [ ]:
for n, p in model.named_parameters():
    print(n, p.shape, p.sum().cpu().detach().numpy(), stat['model'][n].sum().cpu().detach().numpy())
    #print(n, p.shape)
    p = stat['model'][n]

In [ ]:
learn = Learner(data,model)

In [ ]:
# 把模型放到device上，如果使用GPU，则设置使用哪几个GPU
if hasattr(learn.model, "module"): 
    learn.model = model.module

learn.model.to(device);

if device.type=='cuda':
    learn.model = torch.nn.DataParallel(model,device_ids=[0,1])

In [ ]:
learn.model.eval();

## 验证模型的整体表现

In [ ]:
data.train_dl.batch_size


In [ ]:
metrics = [partial(nb_anchors_loss_metrics.clas_L,gaf=gaf),
           partial(nb_anchors_loss_metrics.cent_L,gaf=gaf),
           partial(nb_anchors_loss_metrics.pConf_L,gaf=gaf),
           partial(nb_anchors_loss_metrics.nConf_L,gaf=gaf),
           partial(nb_anchors_loss_metrics.clas_acc,gaf=gaf),
           partial(nb_anchors_loss_metrics_std.cent_d_std,gaf=gaf)]

learn.loss_func = partial(nb_anchors_loss_metrics_std.yolo_L_std, gaf=gaf)

res = learn.validate(data.train_dl,metrics=metrics)

res

## 观察

In [ ]:
classes = data.train_ds.y.classes[1:]
classes

In [ ]:
data.valid_dl.batch_size

In [ ]:
x,y = next(iter(data.valid_dl))
show_gt(x,y)

In [ ]:
len(y)

In [ ]:
y[0].shape,y[1].shape

In [ ]:
show_gt_anchors(x,y, device)

In [ ]:
out = learn.model(x)

In [ ]:
out[0].shape, out[1].shape, out[2].shape

In [ ]:
def dbg():    
    import pdb; pdb.set_trace()    
    pout = nb_interpretation.process_netout(out[0].detach(), out[0].detach(), out[2].detach(), out[1].detach(), gaf)
dbg()    

In [ ]:
# def dbg():    
#     import pdb; pdb.set_trace()    
#     t = netouts2preds_std(out, gaf)
#     return t
# t = dbg()

In [ ]:
#show_gt_anchors(x, [t[0], t[2]], device)

In [ ]:
# def dbg():    
#     import pdb; pdb.set_trace() 
#     show_gt_anchors(x, [t[0], t[2]], device)
# dbg()    

In [ ]:
top_idx=show_top_pred(x,out,20)

In [ ]:
top_idx

In [ ]:
batch_x = [data.denorm(xx) for xx in x]
nb_interpretation.show_batch_pred(batch_x, t[0], t[2], t[1], data.classes)

In [ ]:
def dbg():    
    import pdb; pdb.set_trace()    
    pout = process_netout(out[0].detach(), out[2].detach(), out[1].detach(), gaf)

pout = dbg()  

In [ ]:
pout = process_netout(out[0].detach(), out[2].detach(), out[1].detach(), gaf)

In [ ]:
pout[0].shape,pout[1].shape,pout[2].shape,pout[3].shape

In [ ]:
TH = 0.98

In [ ]:
torch.where(pout[1] > TH)[0].shape

In [ ]:
filt = torch.where(pout[1] > TH)

In [ ]:
boxs, confs, cats, prbs = pout[0], pout[1], pout[2], pout[3]

In [ ]:
cats.shape

In [ ]:
boxs = boxs[filt]

In [ ]:
confs = confs[filt]

In [ ]:
torch.min(confs)

In [ ]:
#pout = nb_interpretation.process_netout(out[0].detach(), [], out[2].detach(), out[1].detach(), gaf)

# export

In [ ]:
!python notebook2script.py --fname 'interpretation_std.ipynb' --outputDir './exp/'